## Spark setting

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz 
!tar xf spark-3.0.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 25 kB/s 
     |████████████████████████████████| 198 kB 53.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=bdf41cdfbaf7b6b528955452c5c1ac2a959511eff18106d2b1822462ff0c0f21
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"

In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [5]:
spark

## Kaggle APIs

________________________________________________________________________

In [6]:
from google.colab import files
files.upload() #upload kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"federicofiorio","key":"3c9fda0dd26cc20d5f652a577743142c"}'}

In [7]:
! pip install kaggle

In [8]:
! mkdir ~/.kaggle

In [9]:
!cp kaggle.json ~/.kaggle/

In [10]:
!ls ~/.kaggle

kaggle.json


In [11]:
!chmod 600 /root/.kaggle/kaggle.json

In [12]:
!kaggle datasets download -d bwandowando/ukraine-russian-crisis-twitter-dataset-1-2-m-rows

100% 7.15G/7.15G [01:21<00:00, 67.2MB/s]
100% 7.15G/7.15G [01:21<00:00, 94.0MB/s]


In [13]:
!ls

kaggle.json
sample_data
spark-3.0.3-bin-hadoop2.7
spark-3.0.3-bin-hadoop2.7.tgz
ukraine-russian-crisis-twitter-dataset-1-2-m-rows.zip


In [14]:
!unzip ukraine-russian-crisis-twitter-dataset-1-2-m-rows.zip

Archive:  ukraine-russian-crisis-twitter-dataset-1-2-m-rows.zip
  inflating: 0401_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0402_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0403_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0404_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0405_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0406_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0407_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0408_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0409_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0410_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0411_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0412_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0413_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0414_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0415_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0416_UkraineCombinedTweetsDeduped.csv.gzip  
  inflat

## Use the data

________________________________________________________________________

In [15]:
import numpy as np
import pandas as pd
import csv
import os
import warnings

In [16]:
filename = r"0401_UkraineCombinedTweetsDeduped.csv.gzip"
df = pd.read_csv(filename, compression='gzip', index_col=0,encoding='utf-8', quoting=csv.QUOTE_ALL)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [17]:
df[0:2].T

,0,1
userid,16882774,3205296069
username,Yaniela,gregffff
acctdesc,"Animal lover, supports those who fight injusti...",NaN
location,Hawaii,NaN
following,1158,122
followers,392,881
totaltweets,88366,99853
usercreatedts,2008-10-21 07:34:04.000000,2015-04-25 11:24:34.000000
tweetid,1509681950042198030,1509681950151348229
tweetcreatedts,2022-04-01 00:00:00.000000,2022-04-01 00:00:00.000000


In [18]:
type(df)

pandas.core.frame.DataFrame

In [19]:
df = df[['tweetid', 'text', 'hashtags']] #keep only 3 columns
df.head()

,tweetid,text,hashtags
0,1509681950042198030,⚡The Ukrainian Air Force would like to address...,[]
1,1509681950151348229,Chernihiv oblast. Ukrainians welcome their lib...,"[{'text': 'russianinvasion', 'indices': [77, 9..."
2,1509681950683926556,America 🇺🇸 is preparing for something worse th...,"[{'text': 'RussianUkrainianWar', 'indices': [7..."
3,1509681951116046336,JUST IN: #Anonymous has hacked &amp; released ...,"[{'text': 'Anonymous', 'indices': [25, 35]}]"
4,1509681951304990720,***PUBLIC MINT NOW LIVE***\n\nFor \n@billionai...,[]


In [20]:
pd.options.display.max_colwidth = 1200
df[df['tweetid']==1509681950151348229		]["hashtags"]

1    [{'text': 'russianinvasion', 'indices': [77, 93]}, {'text': 'StandWithUkraine️', 'indices': [97, 115]}, {'text': 'UkraineUnderAttack', 'indices': [116, 135]}]
Name: hashtags, dtype: object

As we can see in the previous cell, the tweet has the hashtag #StopRussia #UkraineUnderAttaсk BUT the dataframe (1st tuple) sais there are no hashtag in the tweet. We will try to recover form this.

In [ ]:
import time
start_time = time.time()
i=0
for row in df.itertuples():
    hashtags_list=[]
    row=''.join(row.hashtags)
    li = row.split("}, ")
    for elem in li:
      new=elem.partition("'text': '")[2].partition("', 'indices'")[0]
      hashtags_list.append(new)
      hashtags_list = [i for i in hashtags_list if i] #delete the '' hashtag
    df['hashtags'] = df['hashtags'].astype('object')
    df.at[i, 'hashtags'] = hashtags_list
    print(i)
    i=i+1
print(time.time() - start_time)

Output streaming troncato alle ultime 5000 righe.
359876
359877
359878
359879
359880
359881
359882
359883
359884
359885
359886
359887
359888
359889
359890
359891
359892
359893
359894
359895
359896
359897
359898
359899
359900
359901
359902
359903
359904
359905
359906
359907
359908
359909
359910
359911
359912
359913
359914
359915
359916
359917
359918
359919
359920
359921
359922
359923
359924
359925
359926
359927
359928
359929
359930
359931
359932
359933
359934
359935
359936
359937
359938
359939
359940
359941
359942
359943
359944
359945
359946
359947
359948
359949
359950
359951
359952
359953
359954
359955
359956
359957
359958
359959
359960
359961
359962
359963
359964
359965
359966
359967
359968
359969
359970
359971
359972
359973
359974
359975
359976
359977
359978
359979
359980
359981
359982
359983
359984
359985
359986
359987
359988
359989
359990
359991
359992
359993
359994
359995
359996
359997
359998
359999
360000
360001
360002
360003
360004
360005
360006
360007
360008
360009
360010
36001

In [21]:
df

,tweetid,text,hashtags
0,1509681950042198030,⚡The Ukrainian Air Force would like to address misinformation published in multiple Western media outlets regarding the situation in the 🇺🇦 sky and support from our @NATO allies. \nMore in 🧵(1/16)\n#ProtectUАSky #StopRussia #UkraineUnderAttaсk,[]
1,1509681950151348229,Chernihiv oblast. Ukrainians welcome their liberators from #russianinvasion. \n\n#StandWithUkraine️ #UkraineUnderAttack #UkraineWillWin #PutinIsaWarCriminal #StopPutin #RussianUkrainianWar #RussiaGoHome #РоссияСмотри #нетвойне https://t.co/86ixYuEtNb,"[{'text': 'russianinvasion', 'indices': [77, 93]}, {'text': 'StandWithUkraine️', 'indices': [97, 115]}, {'text': 'UkraineUnderAttack', 'indices': [116, 135]}]"
2,1509681950683926556,America 🇺🇸 is preparing for something worse than the #RussianUkrainianWar...Taiwan 🇹🇼 Last month we entered into a new cold war with Russia 🇷🇺 what's the best policy for the future of 🇺🇸🇨🇳 relations over Taiwan? \n\nFull Video👇\nhttps://t.co/W58KdpqSly\n\n#China #Taiwan,"[{'text': 'RussianUkrainianWar', 'indices': [74, 94]}]"
3,1509681951116046336,"JUST IN: #Anonymous has hacked &amp; released 62,000 emails from the Marathon Group, a Russian investment firm owned by oligarch Alexander Vinokurov, currently under EU sanctions. Vinokurov is also the son-in-law of Russian Foreign Minister Lavrov. #OpRussia \n#DDoSecrets https://t.co/kATuo2CxC1","[{'text': 'Anonymous', 'indices': [25, 35]}]"
4,1509681951304990720,"***PUBLIC MINT NOW LIVE***\n\nFor \n@billionaire_Wmn\n\nWin $100,000 during public sale mint. \n\nVisit https://t.co/qev7nrD3my to Mint and for further information! Good Luck!\n#nft #mint https://t.co/54daHYVlAP",[]
...,...,...,...
364870,1510044327510265863,@montreal110 @femeninna je trouve plutôt terrible de ne pas pouvoir les aider... #impuissance #ukraine #russie,"[{'text': 'impuissance', 'indices': [81, 93]}, {'text': 'ukraine', 'indices': [94, 102]}, {'text': 'russie', 'indices': [103, 110]}]"
364871,1510044329993138176,"“From where Winston stood it was just possible to read, picked out on it’s white face in elegant lettering, the three slogans of the Party: \nWAR IS PEACE\nFREEDOM IS SLAVERY\nIGNORANCE IS STRENGTH.”\n\n@POTUS @WhiteHouse \n#Ukraine","[{'text': 'Ukraine', 'indices': [218, 226]}]"
364872,1510044330085347328,"When I said tonight in front of 3000 people In Sioux Falls, South Dakota that @ZelenskyyUa is a a “badass”, the room erupted on thunderous applause! #SlavaUkraini !",[]
364873,1510044333533184005,#1Abr #Ucrania \nVídeo de los helicópteros ucranianos en pleno ataque a depósitos de combustible en Rusia. https://t.co/dTeKrWwyba,"[{'text': '1Abr', 'indices': [0, 5]}, {'text': 'Ucrania', 'indices': [6, 14]}]"


In [22]:
from pyspark.sql.types import *

schema = StructType([StructField("tweetid", StringType(), True)\
                   ,StructField("text", StringType(), True)\
                   ,StructField("hashtags", StringType(), True)])

#create spark dataframe using schema
df_spark = spark.createDataFrame(df,schema=schema)

In [23]:
type(df_spark)

pyspark.sql.dataframe.DataFrame

In [24]:
df_spark.show()

+-------------------+--------------------+--------------------+
|            tweetid|                text|            hashtags|
+-------------------+--------------------+--------------------+
|1509681950042198030|⚡The Ukrainian Ai...|                  []|
|1509681950151348229|Chernihiv oblast....|[{'text': 'russia...|
|1509681950683926556|America 🇺🇸 is p...|[{'text': 'Russia...|
|1509681951116046336|JUST IN: #Anonymo...|[{'text': 'Anonym...|
|1509681951304990720|***PUBLIC MINT NO...|                  []|
|1509681952000937999|The Amazing story...|[{'text': 'Russia...|
|1509681952978210849|&amp;quot;How we ...|                  []|
|1509681953053843466|India's purchase ...|[{'text': 'Russia...|
|1509681953091457035|The most basic te...|[{'text': 'Ukrain...|
|1509681953418711050|"The image that R...|[{'text': 'Putin'...|
|1509681953418752008|#Russia’s Preside...|[{'text': 'Russia...|
|1509681953636700160|Sad to read of th...|[{'text': 'Russia...|
|1509681953947074560|الرئيس الروسي يصع...|

## RDD TEXT SERVE PER PRENDERE GLI HASHTAG DAL TWEET IN QUANTO LA COLONNA HASHTAG DEL DATASET NON è ACCURATA

In [52]:
import nltk
from nltk import word_tokenize
import re
nltk.download('punkt') #to make it work

rdd = df_spark.rdd #to get the rdd from dataframe
rdd.take(10)

rdd_text = rdd.map(lambda x : (x[0], x[1])).map(lambda x:(x[0], word_tokenize(x[1]))) #id, word_tokenized
rdd_text.first()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


('1509681950042198030',
 ['⚡The',
  'Ukrainian',
  'Air',
  'Force',
  'would',
  'like',
  'to',
  'address',
  'misinformation',
  'published',
  'in',
  'multiple',
  'Western',
  'media',
  'outlets',
  'regarding',
  'the',
  'situation',
  'in',
  'the',
  '🇺🇦',
  'sky',
  'and',
  'support',
  'from',
  'our',
  '@',
  'NATO',
  'allies',
  '.',
  'More',
  'in',
  '🧵',
  '(',
  '1/16',
  ')',
  '#',
  'ProtectUАSky',
  '#',
  'StopRussia',
  '#',
  'UkraineUnderAttaсk'])

## ESTRAGGO SOLO GLI HASHTAGS

In [54]:
def processHashtags(tweet):
  hashtags = []
  for i,word in enumerate(tweet):
    if word == '#':
      hashtags.append(re.sub(r'[^\w\s]','',tweet[i+1]).lower()) #remove punctuation with regexp and put them lowercase
  return hashtags

hashtags_per_tweet = rdd_text.map(lambda x: (x[0], processHashtags(x[1])))
hashtags_per_tweet.take(10)

[('1509681950042198030', ['protectuаsky', 'stoprussia', 'ukraineunderattaсk']),
 ('1509681950151348229',
  ['russianinvasion',
   'standwithukraine',
   'ukraineunderattack',
   'ukrainewillwin',
   'putinisawarcriminal',
   'stopputin',
   'russianukrainianwar',
   'russiagohome',
   'россиясмотри',
   'нетвойне']),
 ('1509681950683926556', ['russianukrainianwar', 'china', 'taiwan']),
 ('1509681951116046336', ['anonymous', 'oprussia', 'ddosecrets']),
 ('1509681951304990720', ['nft', 'mint']),
 ('1509681952000937999',
  ['russia',
   'ukraine',
   'motivation',
   'netde',
   'edude',
   'delaware',
   'government',
   'usa']),
 ('1509681952978210849',
  ['ukraine', 'ukrainewar', 'russia', 'ukraineinvasion']),
 ('1509681953053843466', ['russian', 'moscow']),
 ('1509681953091457035', ['ukraine']),
 ('1509681953418711050', ['putin', 'medvedev', 'russia', 'ukraine'])]

<bound method RDD.first of PythonRDD[47] at RDD at PythonRDD.scala:53>

In [ ]:
#devo togliere le parole degli hashtags dal testo,
#a questo punto: l'rdd degli hashtags dovrebbe essere corretto, l'rdd col testo va ancora processato

In [ ]:
#per togliere le parole dal testo devo unire i due rdd hashtags e testo in modo da avere una lista di parole che so che dovrò togliere

In [ ]:
#servirà dopo in caso facessimo anche rdd_testo
import string     
puncts = string.punctuation 
puncts

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

### rdd_hashtags è l'rdd da usare per apriori, prima però andrebbe risolto il problema del reducebykey 

In [ ]:
i=0
hashtags_list=[]
for row in df.itertuples():
    li = row.hashtags.split("}, ")
    for elem in li:
      new=elem.partition("'text': '")[2].partition("', 'indices'")[0]
      hashtags_list.append(new)
    #df['hashtags'] = df['hashtags'].astype('object')
    #df.at[i,'hashtags'] = hashtags_list
    #hashtags_list.clear()
    i=i+1
    if(i==2):
      break
print(i)

AttributeError: ignored

In [ ]:
df3=df
df3

,tweetid,text,hashtags
0,1509681950042198030,⚡The Ukrainian Air Force would like to address misinformation published in multiple Western media outlets regarding the situation in the 🇺🇦 sky and support from our @NATO allies. \nMore in 🧵(1/16)\n#ProtectUАSky #StopRussia #UkraineUnderAttaсk,[]
1,1509681950151348229,Chernihiv oblast. Ukrainians welcome their liberators from #russianinvasion. \n\n#StandWithUkraine️ #UkraineUnderAttack #UkraineWillWin #PutinIsaWarCriminal #StopPutin #RussianUkrainianWar #RussiaGoHome #РоссияСмотри #нетвойне https://t.co/86ixYuEtNb,"[{'text': 'russianinvasion', 'indices': [77, 93]}, {'text': 'StandWithUkraine️', 'indices': [97, 115]}, {'text': 'UkraineUnderAttack', 'indices': [116, 135]}]"
2,1509681950683926556,America 🇺🇸 is preparing for something worse than the #RussianUkrainianWar...Taiwan 🇹🇼 Last month we entered into a new cold war with Russia 🇷🇺 what's the best policy for the future of 🇺🇸🇨🇳 relations over Taiwan? \n\nFull Video👇\nhttps://t.co/W58KdpqSly\n\n#China #Taiwan,"[{'text': 'RussianUkrainianWar', 'indices': [74, 94]}]"
3,1509681951116046336,"JUST IN: #Anonymous has hacked &amp; released 62,000 emails from the Marathon Group, a Russian investment firm owned by oligarch Alexander Vinokurov, currently under EU sanctions. Vinokurov is also the son-in-law of Russian Foreign Minister Lavrov. #OpRussia \n#DDoSecrets https://t.co/kATuo2CxC1","[{'text': 'Anonymous', 'indices': [25, 35]}]"
4,1509681951304990720,"***PUBLIC MINT NOW LIVE***\n\nFor \n@billionaire_Wmn\n\nWin $100,000 during public sale mint. \n\nVisit https://t.co/qev7nrD3my to Mint and for further information! Good Luck!\n#nft #mint https://t.co/54daHYVlAP",[]
...,...,...,...
364870,1510044327510265863,@montreal110 @femeninna je trouve plutôt terrible de ne pas pouvoir les aider... #impuissance #ukraine #russie,"[{'text': 'impuissance', 'indices': [81, 93]}, {'text': 'ukraine', 'indices': [94, 102]}, {'text': 'russie', 'indices': [103, 110]}]"
364871,1510044329993138176,"“From where Winston stood it was just possible to read, picked out on it’s white face in elegant lettering, the three slogans of the Party: \nWAR IS PEACE\nFREEDOM IS SLAVERY\nIGNORANCE IS STRENGTH.”\n\n@POTUS @WhiteHouse \n#Ukraine","[{'text': 'Ukraine', 'indices': [218, 226]}]"
364872,1510044330085347328,"When I said tonight in front of 3000 people In Sioux Falls, South Dakota that @ZelenskyyUa is a a “badass”, the room erupted on thunderous applause! #SlavaUkraini !",[]
364873,1510044333533184005,#1Abr #Ucrania \nVídeo de los helicópteros ucranianos en pleno ataque a depósitos de combustible en Rusia. https://t.co/dTeKrWwyba,"[{'text': '1Abr', 'indices': [0, 5]}, {'text': 'Ucrania', 'indices': [6, 14]}]"


In [ ]:
df['hashtags'] = df['hashtags'].astype('object')
df.at[1, 'hashtags'] = [1, 2, 3]
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,tweetid,text,hashtags
0,1509681950042198030,⚡The Ukrainian Air Force would like to address misinformation published in multiple Western media outlets regarding the situation in the 🇺🇦 sky and support from our @NATO allies. \nMore in 🧵(1/16)\n#ProtectUАSky #StopRussia #UkraineUnderAttaсk,[]
1,1509681950151348229,Chernihiv oblast. Ukrainians welcome their liberators from #russianinvasion. \n\n#StandWithUkraine️ #UkraineUnderAttack #UkraineWillWin #PutinIsaWarCriminal #StopPutin #RussianUkrainianWar #RussiaGoHome #РоссияСмотри #нетвойне https://t.co/86ixYuEtNb,"[1, 2, 3]"
2,1509681950683926556,America 🇺🇸 is preparing for something worse than the #RussianUkrainianWar...Taiwan 🇹🇼 Last month we entered into a new cold war with Russia 🇷🇺 what's the best policy for the future of 🇺🇸🇨🇳 relations over Taiwan? \n\nFull Video👇\nhttps://t.co/W58KdpqSly\n\n#China #Taiwan,"[{'text': 'RussianUkrainianWar', 'indices': [74, 94]}]"
3,1509681951116046336,"JUST IN: #Anonymous has hacked &amp; released 62,000 emails from the Marathon Group, a Russian investment firm owned by oligarch Alexander Vinokurov, currently under EU sanctions. Vinokurov is also the son-in-law of Russian Foreign Minister Lavrov. #OpRussia \n#DDoSecrets https://t.co/kATuo2CxC1","[{'text': 'Anonymous', 'indices': [25, 35]}]"
4,1509681951304990720,"***PUBLIC MINT NOW LIVE***\n\nFor \n@billionaire_Wmn\n\nWin $100,000 during public sale mint. \n\nVisit https://t.co/qev7nrD3my to Mint and for further information! Good Luck!\n#nft #mint https://t.co/54daHYVlAP",[]
...,...,...,...
364870,1510044327510265863,@montreal110 @femeninna je trouve plutôt terrible de ne pas pouvoir les aider... #impuissance #ukraine #russie,"[{'text': 'impuissance', 'indices': [81, 93]}, {'text': 'ukraine', 'indices': [94, 102]}, {'text': 'russie', 'indices': [103, 110]}]"
364871,1510044329993138176,"“From where Winston stood it was just possible to read, picked out on it’s white face in elegant lettering, the three slogans of the Party: \nWAR IS PEACE\nFREEDOM IS SLAVERY\nIGNORANCE IS STRENGTH.”\n\n@POTUS @WhiteHouse \n#Ukraine","[{'text': 'Ukraine', 'indices': [218, 226]}]"
364872,1510044330085347328,"When I said tonight in front of 3000 people In Sioux Falls, South Dakota that @ZelenskyyUa is a a “badass”, the room erupted on thunderous applause! #SlavaUkraini !",[]
364873,1510044333533184005,#1Abr #Ucrania \nVídeo de los helicópteros ucranianos en pleno ataque a depósitos de combustible en Rusia. https://t.co/dTeKrWwyba,"[{'text': '1Abr', 'indices': [0, 5]}, {'text': 'Ucrania', 'indices': [6, 14]}]"


## in caso servano
__________________________________________________________________

In [ ]:
#possibile farlo con spark credo per parallelizzare
rows_eng = df.language == 'en'
rows_selected = []

for i,r in enumerate(rows_eng):
  if r == True:
    rows_selected.append(df.iloc[i])

eng_df = pd.DataFrame(rows_selected)

AttributeError: ignored

In [ ]:
#Creating dictionary for words and dictionary for hashtags
from collections import defaultdict

hashtags = defaultdict(lambda : [])
texts = defaultdict(lambda : [])

In [ ]:
for index, row in eng_df.iterrows():
  texts[row.userid].append(row.text)
  hashtags[row.userid].append(row.hashtags)

In [ ]:
print(texts[1054170412194553857])
print("_________________________________________________")
print(hashtags[1054170412194553857])

['“From where Winston stood it was just possible to read, picked out on it’s white face in elegant lettering, the three slogans of the Party: \nWAR IS PEACE\nFREEDOM IS SLAVERY\nIGNORANCE IS STRENGTH.”\n\n@POTUS @WhiteHouse \n#Ukraine']
_________________________________________________
["[{'text': 'Ukraine', 'indices': [218, 226]}]"]


In [ ]:
string="[{'text': 'russianinvasion', 'indices': [77, 93]"

In [ ]:
string.partition("'text': '")[2].partition("', 'indices'")[0]

'russianinvasion'

In [ ]:
x=row.hashtags
x

"[{'text': 'russianinvasion', 'indices': [77, 93]}, {'text': 'StandWithUkraine️', 'indices': [97, 115]}, {'text': 'UkraineUnderAttack', 'indices': [116, 135]}]"

In [ ]:
li = list(x.split("}, "))
li

["[{'text': 'russianinvasion', 'indices': [77, 93]",
 "{'text': 'StandWithUkraine️', 'indices': [97, 115]",
 "{'text': 'UkraineUnderAttack', 'indices': [116, 135]}]"]

matto qua parte inerente a rdd_text (non si sa mai)

In [ ]:
#take only english texts
rdd_text =  rdd.map(lambda x : (x[0], x[1]))
rdd_text.take(10)

[('1509681950042198030',
  '⚡The Ukrainian Air Force would like to address misinformation published in multiple Western media outlets regarding the situation in the 🇺🇦 sky and support from our @NATO allies. \nMore in 🧵(1/16)\n#ProtectUАSky #StopRussia #UkraineUnderAttaсk'),
 ('1509681950151348229',
  'Chernihiv oblast. Ukrainians welcome their liberators from #russianinvasion. \n\n#StandWithUkraine️ #UkraineUnderAttack #UkraineWillWin #PutinIsaWarCriminal #StopPutin #RussianUkrainianWar #RussiaGoHome #РоссияСмотри #нетвойне https://t.co/86ixYuEtNb'),
 ('1509681950683926556',
  "America 🇺🇸 is preparing for something worse than the #RussianUkrainianWar...Taiwan 🇹🇼 Last month we entered into a new cold war with Russia 🇷🇺 what's the best policy for the future of 🇺🇸🇨🇳 relations over Taiwan? \n\nFull Video👇\nhttps://t.co/W58KdpqSly\n\n#China #Taiwan"),
 ('1509681951116046336',
  'JUST IN: #Anonymous has hacked &amp; released 62,000 emails from the Marathon Group, a Russian investment firm ow

In [ ]:
import nltk
from nltk import word_tokenize
nltk.download('punkt') #to make it work

#ok l'idea è prendere gli hashtag e la parola subito dopo e transportarli nel l'elenco degli hashtags in modo da avere dei dati corretti
#dopo aver preso gli hashtags levo la punteggiatura con string.punctuation che tanto non è rilevante per i baskets

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
rdd_text = rdd_text.map(lambda x:(x[0], word_tokenize(x[1]))) #id, word_tokenized
rdd_text.first()